In [2]:
import re
from wordcloud import WordCloud
import pandas as pd
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from
from flask import Flask, jsonify
import sqlite3
import demoji
import emoji
from unidecode import unidecode
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter

## FUNGSI-FUNGSI

In [30]:
#fungsi untuk menghilangkan karakter
def clean_karakter(texts):
    tanpa_hex = [re.sub(r'\\x..', ' ', text) for text in texts]
    tanpa_backslash = [re.sub(r'\\', ' ', text) for text in tanpa_hex]
    tanpa_newline = [re.sub(r'\n', ' ', text) for text in tanpa_backslash]
    non_ascii = [re.sub(r'[^\x00-\x7F]+', ' ', text) for text in tanpa_newline]   
    non_url = [re.sub(r'https?://\S+|www\.\S+', ' ', text) for text in non_ascii]
    non_whitespace = [re.sub(r'\s+', ' ', text) for text in non_url]
    only_text = [re.sub(r'[^a-zA-Z0-9\s]', ' ', text) for text in non_whitespace]
    lower_text = [re.sub(r'[A-Z]', lambda x: x.group(0).lower(), text) for text in only_text]
    tanpa_kata_user = [re.sub(r'\buser\b', ' ', text) for text in lower_text]
    tanpa_kata_RT = [re.sub(r'\brt\b', ' ', text) for text in tanpa_kata_user]
    cleaned_text_new = [re.sub(r'\s+', ' ', text).strip() for text in tanpa_kata_RT]
    return cleaned_text_new

In [ ]:
def remove_abusive(text):
    df_abusive = pd.read_csv('asset-challenges/abusive.csv')
    abusive_words = df_abusive['ABUSIVE'].tolist()
    abusive_words_lower = [word.lower() for word in abusive_words]
    for kata in abusive_words_lower:
        text = re.sub(r'\b' + re.escape(kata) + r'\b', '', text, flags=re.IGNORECASE)
    cleaned_text = re.sub(' +', ' ', text).strip()
    
    return cleaned_text

In [ ]:
def replace_kata_alay(text, df_kamusalay):
    words = text.split()
    for i in range(len(words)):
        if words[i] in df_kamusalay['kata_alay'].values:
            new_word = df_kamusalay.loc[df_kamusalay['kata_alay'] == words[i], 'kata_baku'].values[0]
            words[i] = new_word
    return ' '.join(words)

# PREPARE MEMBUAT API

In [38]:
def clean_kar(text):
    text = re.sub(r'\\x..', ' ', text)
    text = re.sub(r'\\', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'[A-Z]', lambda x: x.group(0).lower(), text)
    text = re.sub(r'\buser\b', ' ', text)
    text = re.sub(r'\brt\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer
# Definisikan parameter untuk feature extraction
max_features = 100000
tokenizer = Tokenizer (num_words=max_features, split = ' ', lower = True)

# Definisikan label untuk sentiment
sentiment = ['negative', 'neutral','positive']

In [40]:
file = open('platinum/resources_of_lstm/x_pad_sequences.pickle', 'rb')
feature_file_from_lstm = pickle.load(file)
file.close()

In [41]:
model_file_from_lstm = load_model ('platinum/model_of_lstm/model.h5')

In [42]:
import pickle

# Memuat kembali model dari berkas
with open("platinum/model_of_nn/model_mlp_classifier.pkl", "rb") as file:
    loaded_model = pickle.load(file)

C:\Users\akung\anaconda3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\akung\anaconda3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [43]:
import pickle

# Buka file untuk dibaca dalam mode biner ("rb")
with open("platinum/feature.p", "rb") as file:
    # Mengembalikan objek yang disimpan menggunakan pickle
    count_vect = pickle.load(file)

C:\Users\akung\anaconda3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# PREPARE MEMBUAT API

In [3]:
app = Flask(__name__)

app.json_encoder = LazyJSONEncoder
swagger_template = {
    "info": {
        "title":  "API Documentation for Data Processing and Modeling",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling"
    },
    "host": "127.0.0.1:5000"
}






# swagger_template = dict(
# info = {
#     'title': LazyString(lambda: 'API Documentation for Deep Learning'),
#     'version': LazyString(lambda: '1.0.0'),
#     'description': LazyString(lambda: 'Dokumentasi API untuk Deep Learning'),
#     },
#     host = LazyString(lambda: request.host)
# )

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'api',
            "route": '/api.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/api/"
}


swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)






# input kalimat
@swag_from("api/neural_network.yml", methods=['POST'])
@app.route('/neural-network', methods=['POST'])
def text_processing_nn():

    original_text = request.form.get('text')

    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', original_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text).strip()
    
#     cleaned_text = replace_kata_alay(cleaned_text, df_kamusalay)
#     cleaned_text = remove_abusive(cleaned_text)
#     insert_data(text, cleaned_text)

    # Mengubah teks menjadi vektor fitur
    text = count_vect.transform([cleaned_text])

    # Menggunakan model untuk memprediksi sentimen teks
    result = loaded_model.predict(text)[0]

    
    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan Neural Network",
        'text': cleaned_text,
        'sentiment': result
    }

    response_data = jsonify(json_response)
    return response_data


@swag_from("api/lstm.yml", methods=['POST'])
@app.route('/lstm', methods=['POST'])
def text_processing_lstm():

    original_text = request.form.get('text')

    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', original_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text).strip()
    
#     cleaned_text = replace_kata_alay(cleaned_text, df_kamusalay)
#     cleaned_text = remove_abusive(cleaned_text)
#     insert_data(text, cleaned_text)

    feature = tokenizer.texts_to_sequences(text)
    feature = pad_sequences(feature, maxlen=feature_file_from_lstm.shape[1])
    
    prediction = model_file_from_lstm.predict(feature)
    get_sentiment = sentiment[np.argmax(prediction[0])]

    
    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan LSTM",
        'data':{
            'text': original_text,
            'sentiment': get_sentiment
        }
    }

    response_data = jsonify(json_response)
    return response_data





# #input file
# @swag_from("C://Users/akung/api/neural_networkfile.yml", methods=['POST'])
# @app.route('/neural_networkfile', methods=['POST'])
# def file_nn():

#     file = request.files.getlist('file')[0]
#     df_file = pd.read_csv(file,encoding='utf-8')
#     df_file = df_file.drop_duplicates()
#     alay_text = clean_karakter(df_file)

    
#     # cleaned_text_new  = [replace_kata_alay(text, df_kamusalay) for text in alay_text]
    
#     # for i in range(len(cleaned_text_new)):
#     #     cleaned_text_new[i] = remove_abusive(cleaned_text_new[i])
    
#     for text in alay_text.iloc[:,0]:
#         feature = tokenizer.texts_to_sequences(text)
#         feature = pad_sequences(feature, maxlen=feature_file_from_lstm.shape[1])
        
#         prediction = model_file_from_lstm.predict(feature)
#         get_sentiment = sentiment[np.argmax(prediction[0])]
    
#         sentiment_result.append({
#             'text': original_text,
#             'sentiment': get_sentiment


#         })
    

#     json_response = {
#                 'status_code': 200,
#                 'description': "Hasil Sentimen analisis menggunakan LSTM",
#                 'data':sentiment_result
#             }
        
#     response_data = jsonify(json_response)
#     return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\akung\AppData\Local\Temp\ipykernel_22296\2218998922.py:3: DeprecationWarning: 'app.json_encoder' is deprecated and will be removed in Flask 2.3. Customize 'app.json_provider_class' or 'app.json' instead.
  app.json_encoder = LazyJSONEncoder
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Mar/2024 19:38:18] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2024 19:38:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2024 19:38:22] "GET /api/ HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2024 19:38:22] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2024 19:38:22] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2024 19:38:22] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2024 19:38:22] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
ERROR:root:jsonify failure; defaulting to json.dumps
Traceback (most recent call last):
  File "C:\Users\akung\a

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------